In [ ]:
# Install the required packages
%pip install transformers symspellpy

from transformers import T5ForConditionalGeneration, T5Tokenizer
from symspellpy import SymSpell, Verbosity
import pkg_resources

In [ ]:

# Load Pre-trained Grammar Correction Model
model_name = "vennify/t5-base-grammar-correction"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# Load SymSpell for Spelling Correction
symspell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt"
)
symspell.load_dictionary(dictionary_path, term_index=0, count_index=1)

In [ ]:
def correct_spelling(text):
    """Fix spelling errors."""
    words = text.split()
    corrected_words = []
    for word in words:
        suggestions = symspell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        corrected_word = suggestions[0].term if suggestions else word
        corrected_words.append(corrected_word)
    return " ".join(corrected_words)

In [ ]:
def correct_grammar(text):
    """Fix grammar errors."""
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=len(text.split()) + 10)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Interactive Testing
input_text = "She go to the park evryday."
print(f"Original: {input_text}")

In [ ]:
# Fix spelling first
spelling_fixed = correct_spelling(input_text)
print(f"Spelling Fixed: {spelling_fixed}")

In [ ]:
# Fix grammar next
grammar_fixed = correct_grammar(spelling_fixed)
print(f"Final Correction: {grammar_fixed}")